In [1]:
import cv2
import os
import time
import imutils
import random
import numpy as np
from skimage import io as iio
from tqdm import tqdm
import random
import shutil

def add_noise(src, num):
    img = src.copy()
    (h, w) = img.shape[:2]
    for i in range(num):
       x = np.random.randint(1, h-1)
       y = np.random.randint(1, w-1)
       img[x-1:x+1, y-1:y+1] = 0
    return img

def dilate(src, shape, dilation_size):
    element = cv2.getStructuringElement(shape, (dilation_size, dilation_size))
    result = cv2.erode(src, element) # Note: The font color is black, so we use erode instead.
    return result

def erode(src, shape, erosion_size):
    element = cv2.getStructuringElement(shape, (erosion_size, erosion_size))
    result = cv2.dilate(src, element) # Note: The font color is black, so we use dilate instead.
    return result




In [2]:
# input_dir = 'C:/Users/bbb50/Desktop/Python/OCR/font_create_ood'
# output_dir = 'C:/Users/bbb50/Desktop/Python/OCR/ai_free_od_20'

In [3]:
# n = 61325
# for char in tqdm(os.listdir(input_dir)):
#     sub_dir = os.path.join(input_dir, char)
#     for img in os.listdir(sub_dir):
#         src = os.path.join(sub_dir, img)
#         dst = os.path.join(output_dir, str(n)+'.png')
#         os.rename(src, dst)
#         n += 1

In [4]:
n = 61325

61325

In [15]:
input_dir = 'C:/Users/bbb50/Desktop/Python/t_brain_new_gray_augmented/t_brain_new_gray_augmented/val'
output_dir = 'C:/Users/bbb50/Desktop/Python/t_brain_new_gray_augmented/t_brain_new_gray_augmented/test'
new_width = 224
new_height = 224
dilation_ksize = 5
erosion_ksize = 5
src_tri = np.array([[27.5,27.5], [110.0,27.5], [27.5,110.0]]).astype(np.float32)
dst_tri = np.array([[5.5,55.0], [110.0,27.5], [5.5,137.5]]).astype(np.float32)
sample_size_per_dir = 800000
nChars = 800


In [9]:
# char = os.listdir(input_dir)[0]
# images = []
# sub_dir = os.path.join(input_dir, char)
# for i in os.listdir(sub_dir):
#     image_name_abs = os.path.join(sub_dir, i)
# #         image = iio.imread(image_name_abs)
#     print(image_name_abs)

C:/Users/bbb50/Desktop/Python/t_brain_new_gray_augmented/t_brain_new_gray_augmented/val\0_宋\0.png
C:/Users/bbb50/Desktop/Python/t_brain_new_gray_augmented/t_brain_new_gray_augmented/val\0_宋\1.png
C:/Users/bbb50/Desktop/Python/t_brain_new_gray_augmented/t_brain_new_gray_augmented/val\0_宋\2.png
C:/Users/bbb50/Desktop/Python/t_brain_new_gray_augmented/t_brain_new_gray_augmented/val\0_宋\3.png
C:/Users/bbb50/Desktop/Python/t_brain_new_gray_augmented/t_brain_new_gray_augmented/val\0_宋\4.png


In [5]:
image_name_abs = 'C:/Users/bbb50/Desktop/Python/t_brain_new_gray_augmented/t_brain_new_gray_augmented/val\\15_川\\2.png'

In [ ]:
for i, img in enumerate(images):
    output_filename = os.path.join(output_dir, char , str(n)+ '.png' )
    stat = cv2.imwrite(output_filename, img)
    n += 1                               
    if stat is False:
        print("Failed to write image %s" % output_filename)

In [ ]:

            output_filename = os.path.join(output_dir, char , str(n)+ '.png' )
            cv2.imencode('.png', img)[1].tofile(output_filename)

In [18]:


for char in tqdm(sorted(os.listdir(input_dir))):
    n = 0
    images = []
    sub_dir = os.path.join(input_dir, char)
    for i in os.listdir(sub_dir):
        image_name_abs = os.path.join(sub_dir, i)
        image = iio.imread(image_name_abs)
        if len(list(image.shape)) != 3:
            image = cv2.imdecode(np.fromfile(file=image_name_abs, dtype=np.uint8), cv2.IMREAD_COLOR)
        # Try cv2.imread() if iio.imread fails
        # image = cv2.imread(image_name_abs, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print("Unable to read image %s" % image_name_abs)
            exit()

        # Converts the input image to the desired size
        h, w, c = image.shape
        if c == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if w != new_width or h != new_height:
            image = cv2.resize(image, (new_width, new_height))

        # Original image
        images.append(image)

        # Rotation
        for angle in np.arange(6, 13, 6):
            inverted = cv2.bitwise_not(image)
            # CCW
            rotated = cv2.bitwise_not(imutils.rotate(inverted, angle))
            images.append(rotated)

            # CW
            rotated = cv2.bitwise_not(imutils.rotate(inverted, 360 - angle))
            images.append(rotated)

        # Dilation
        dilated = dilate(image, cv2.MORPH_CROSS, dilation_ksize)
        images.append(dilated)

        # Erosion
        if erosion_ksize != 0:
            eroded = erode(image, cv2.MORPH_CROSS, erosion_ksize)
            images.append(eroded)

        # Adds noises to the original image
        noised = add_noise(image, 50)
        images.append(noised)

        # Adds noises to the dilated/eroded images
        dilated_noised = add_noise(dilated, 50)
        if erosion_ksize != 0:
            eroded_noised = add_noise(eroded, 50)

        # Transformation
        warp_mat = cv2.getAffineTransform(src_tri, dst_tri)
        transformed = cv2.warpAffine(image, warp_mat, (image.shape[1], image.shape[0]), borderValue=(255,255,255))
        images.append(transformed)

        # Transforms the dilated image
        dilated_transformed = cv2.warpAffine(dilated, warp_mat, (image.shape[1], image.shape[0]), borderValue=(255,255,255))
        images.append(dilated_transformed)

        # Transforms the eroded image
        if erosion_ksize != 0:
            eroded_transformed = cv2.warpAffine(eroded, warp_mat, (image.shape[1], image.shape[0]), borderValue=(255,255,255))
            images.append(eroded_transformed)

        # Transforms the dilated-noised/eroded-noised images
        dilated_noised_transformed = cv2.warpAffine(dilated_noised, warp_mat, (image.shape[1], image.shape[0]), borderValue=(255,255,255))
        if erosion_ksize != 0:
            eroded_noised_transformed = cv2.warpAffine(eroded_noised, warp_mat, (image.shape[1], image.shape[0]), borderValue=(255,255,255))

        # Blurs the input image
        blurred = cv2.GaussianBlur(image, (5,5), 0)
        images.append(blurred)

        # Blurs the noised image
        noised_blurred = cv2.GaussianBlur(noised, (5,5), 0)
        images.append(noised_blurred)

        # Blurs the dilated, noised, transformed image
        dilated_noised_transformed_blurred = cv2.GaussianBlur(dilated_noised_transformed, (5,5), 0)
        images.append(dilated_noised_transformed_blurred)

        # Blurs the eroded, noised, transformed image
        if erosion_ksize != 0:
            eroded_noised_transformed_blurred = cv2.GaussianBlur(eroded_noised_transformed, (5,5), 0)
            images.append(eroded_noised_transformed_blurred)

        for i, img in enumerate(images):
            output_filename = os.path.join(output_dir, char , str(n)+ '.png' )
            cv2.imencode('.png', img)[1].tofile(output_filename)

            n += 1                               


 97%|█████████████████████████████████████████████████████████████████████████████▊  | 778/800 [01:57<00:03,  6.61it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/bbb50/Desktop/Python/t_brain_new_gray_augmented/t_brain_new_gray_augmented/test\\800_isnull\\0.png'

In [17]:
images

[array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 25

In [27]:
tar_dir = 'C:/Users/bbb50/Desktop/Python/OCR/ai_free_od_20_aug'

In [28]:
dst_dir = 'C:/Users/bbb50/Desktop/Python/OCR/ai_free_od_20_aug_train'

In [32]:
sample_list = random.sample(os.listdir(tar_dir), 854120)
n = 0
for i in tqdm(sample_list):
    src = os.path.join(tar_dir, i)
    dst = os.path.join(dst_dir, str(n)+'.png')
    os.rename(src, dst)
    n += 1

100%|████████████████████████████████████████████████████████████████████████| 854120/854120 [11:21<00:00, 1254.20it/s]


In [ ]:
854120

In [18]:
n = 0
for char in os.listdir(tar_dir):
    sub_dir = os.path.join(tar_dir, char)
    n += len(os.listdir(sub_dir))

In [2]:
src_dir = 'C:/Users/bbb50/Desktop/Python/OCR/ai_free_od_augmentation_train/ai_free_od_augmentation_train'
dst_dir = 'C:/Users/bbb50/Desktop/Python/OCR/ai_free_od_augmentation_train/ai_free_od_augmentation_train_2'

In [3]:

n = 0
for name in tqdm(os.listdir(src_dir)):    
    src = os.path.join(src_dir, name)
    dst = os.path.join(dst_dir, str(n)+'.png')
    os.rename(src, dst)
    n += 1

100%|████████████████████████████████████████████████████████████████████████| 548721/548721 [05:03<00:00, 1806.07it/s]


In [11]:
tar_dir = 'C:/Users/bbb50/Desktop/Python/OCR/ai_free_od_augmentation_train/ai_free_od_augmentation_train_2'

In [12]:
img_amount = len(os.listdir(tar_dir))

In [13]:
dif = 854120 - img_amount

In [14]:
n = img_amount

In [15]:
sample_list = random.sample(range(len(os.listdir(tar_dir))), dif)

In [16]:
for i in tqdm(sample_list):
    src = os.path.join(tar_dir, str(i) + '.png')
    dst = os.path.join(tar_dir, str(n) + '.png')
    shutil.copyfile(src, dst)
    n += 1

100%|█████████████████████████████████████████████████████████████████████████| 305399/305399 [05:46<00:00, 882.42it/s]
